### Chapter6_Queation 9(a)~(e)

In this exercise, we will predict the number of applications received using the other variables in the <font color="brown">College</font> data set.

In [32]:
from ISLP import load_data
df=load_data("College")
print(df.head())

  Private  Apps  Accept  Enroll  Top10perc  Top25perc  F.Undergrad  \
0     Yes  1660    1232     721         23         52         2885   
1     Yes  2186    1924     512         16         29         2683   
2     Yes  1428    1097     336         22         50         1036   
3     Yes   417     349     137         60         89          510   
4     Yes   193     146      55         16         44          249   

   P.Undergrad  Outstate  Room.Board  Books  Personal  PhD  Terminal  \
0          537      7440        3300    450      2200   70        78   
1         1227     12280        6450    750      1500   29        30   
2           99     11250        3750    400      1165   53        66   
3           63     12960        5450    450       875   92        97   
4          869      7560        4120    800      1500   76        72   

   S.F.Ratio  perc.alumni  Expend  Grad.Rate  
0       18.1           12    7041         60  
1       12.2           16   10527         56  
2    

(a) Split the data set into a training set and a test set.

In [33]:
print(df[df.isna().any(axis=1)])
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=314657018)

print("\ntrain_df",train_df.shape)
print(train_df.head())
print("\ntest_df",test_df.shape)
print(test_df.head())

Empty DataFrame
Columns: [Private, Apps, Accept, Enroll, Top10perc, Top25perc, F.Undergrad, P.Undergrad, Outstate, Room.Board, Books, Personal, PhD, Terminal, S.F.Ratio, perc.alumni, Expend, Grad.Rate]
Index: []

train_df (621, 18)
    Private  Apps  Accept  Enroll  Top10perc  Top25perc  F.Undergrad  \
163     Yes  1994    1656     495         50         80         1983   
590     Yes   759     729     244         57         81          867   
716     Yes   647     518     271         17         43         1014   
354     Yes  6411    2140    1078         96         99         4481   
166     Yes  1998    1376     651         41         88         1539   

     P.Undergrad  Outstate  Room.Board  Books  Personal  PhD  Terminal  \
163           36     14300        5020    550       950   78        94   
590           51     10900        4450    500      1000   81        91   
716          387      9140        3365    500      2245   51        65   
354           28     20100        5975 

(b) Fit a linear model using least squares on the training set, and report the test error obtained.

In [34]:
# (b) Fit a linear model (least squares) on the training set and report test error
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Target and predictors
y_train = train_df['Apps']
y_test = test_df['Apps']
X_train = train_df.drop(columns=['Apps'])
X_test = test_df.drop(columns=['Apps'])

# Convert categorical variables to dummies (drop_first to avoid singularities)
X_train = pd.get_dummies(X_train, drop_first=True)
X_test = pd.get_dummies(X_test, drop_first=True)
# Ensure test set has same columns as train set (missing cols -> 0)
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

# Fit linear regression (ordinary least squares)
lm = LinearRegression()
lm.fit(X_train, y_train)

# Predict on test set and report error
y_pred = lm.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
print(f"Test MSE: {mse:.4f}")
print(f"Test RMSE: {rmse:.4f}")

# Optionally show first 5 predictions vs true values
print("\nFirst 5 true vs predicted Apps:\n")
print(pd.DataFrame({'Apps_true': y_test.values, 'Apps_pred': y_pred}).head())

Test MSE: 750808.3714
Test RMSE: 866.4920

First 5 true vs predicted Apps:

   Apps_true    Apps_pred
0        484   139.538127
1       6502  4560.322780
2       4186  4247.880930
3        261    78.481369
4        798   924.758329


(c) Fit a ridge regression model on the training set, with $\lambda$ chosen by cross-validation. Report the test error obtained.

In [42]:
# (c) Ridge regression with lambda chosen by cross-validation (RidgeCV)
import numpy as np
from sklearn.linear_model import RidgeCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# Reuse X_train/X_test prepared earlier (after get_dummies)
# Standardize features for regularized methods
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

alphas = np.logspace(-6, 6, 25)
ridge_cv = RidgeCV(alphas=alphas, scoring='neg_mean_squared_error', cv=5)
ridge_cv.fit(X_train_scaled, y_train)
best_alpha = ridge_cv.alpha_
y_pred_ridge = ridge_cv.predict(X_test_scaled)
mse_ridge = mean_squared_error(y_test, y_pred_ridge)
rmse_ridge = mse_ridge ** 0.5
print(f"Ridge best alpha: {best_alpha}")
print(f"Ridge Test MSE: {mse_ridge:.4f}")
print(f"Ridge Test RMSE: {rmse_ridge:.4f}")

Ridge best alpha: 1e-06
Ridge Test MSE: 750808.3746
Ridge Test RMSE: 866.4920


(d) Fit a lasso model on the training set, with $\lambda$ chosen by cross-validation. Report the test error obtained, along with the number of non-zero coefficient estimates.

In [36]:
# (d) Lasso with cross-validation; report test error and number of non-zero coefficients
from sklearn.linear_model import LassoCV

# Use the scaled features from above (X_train_scaled, X_test_scaled)
lasso_cv = LassoCV(alphas=None, cv=5, max_iter=10000, random_state=0)
lasso_cv.fit(X_train_scaled, y_train)
best_alpha_lasso = lasso_cv.alpha_
y_pred_lasso = lasso_cv.predict(X_test_scaled)
mse_lasso = mean_squared_error(y_test, y_pred_lasso)
rmse_lasso = mse_lasso ** 0.5
n_nonzero = (lasso_cv.coef_ != 0).sum()
print(f"Lasso best alpha: {best_alpha_lasso}")
print(f"Lasso Test MSE: {mse_lasso:.4f}")
print(f"Lasso Test RMSE: {rmse_lasso:.4f}")
print(f"Number of non-zero coefficients: {n_nonzero}")

Lasso best alpha: 18.893744002000265
Lasso Test MSE: 771411.1084
Lasso Test RMSE: 878.3001
Number of non-zero coefficients: 14


/Users/andyw.p.chen/Documents/Project/2025_Fraud-Detection-project/ML/virtual/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:1641: FutureWarning: 'alphas=None' is deprecated and will be removed in 1.9, at which point the default value will be set to 100. Set 'alphas=100' to silence this warning.
  warnings.warn(


(e) Fit a PCR model on the training set, with $M$ chosen by cross-validation. Report the test error obtained, along with the value of $M$ selected by cross-validation.

In [37]:
# (e) PCR: choose M by cross-validation and report test error and selected M
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score

# Compute PCA on scaled training data
pca = PCA()
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)
n_components = X_train_pca.shape[1]
cv_scores = []
for m in range(1, n_components + 1):
    X_m = X_train_pca[:, :m]
    scores = cross_val_score(LinearRegression(), X_m, y_train, cv=5, scoring='neg_mean_squared_error')
    cv_scores.append(scores.mean())
best_m = int(np.argmax(cv_scores) + 1)
# Fit final model with best_m components
lm_pcr = LinearRegression()
lm_pcr.fit(X_train_pca[:, :best_m], y_train)
y_pred_pcr = lm_pcr.predict(X_test_pca[:, :best_m])
mse_pcr = mean_squared_error(y_test, y_pred_pcr)
rmse_pcr = mse_pcr ** 0.5
print(f"PCR selected M: {best_m}")
print(f"PCR Test MSE: {mse_pcr:.4f}")
print(f"PCR Test RMSE: {rmse_pcr:.4f}")

PCR selected M: 17
PCR Test MSE: 750808.3714
PCR Test RMSE: 866.4920
